# Notebook to Implement Model Training - LGBM

---

### 1) Setup

In [34]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy import stats
from scipy.stats import wilcoxon

from sklearn.metrics import balanced_accuracy_score, make_scorer, confusion_matrix, classification_report
from sklearn.model_selection import RepeatedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import optuna
import lightgbm as lgb

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer


from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [128]:
SEED = 42

In [129]:
TRAIN_CLINICAL_FILENAME = "train_set2.csv"
TEST_CLINICAL_FILENAME = "test_set2.csv"

---

### 2) Read and Preprocess Data

In [151]:
train = pd.read_csv(TRAIN_CLINICAL_FILENAME, sep=",")
train = train.iloc[: , 1:]

In [152]:
train.shape

(132, 547)

In [153]:
train.head()

,Freq.1198.26559877594,Freq.1204.07148226988,Freq.1211.13623593336,Freq.1217.73683596229,Freq.1223.16958659828,Freq.1234.45996741201,Freq.1239.41043347512,Freq.1244.8230732813,Freq.1254.64710066874,Freq.1261.3335177338,...,Freq.9059.04784969308,Freq.9098.3102509794,Freq.9436.045671808,Freq.9594.78353572215,Freq.9799.45047805923,Freq.10431.4344537929,Freq.11007.128608859,Freq.11035.3843513154,Freq.11160.836582373,Group
0,0.000251,0.000345,0.000074,3.541783e-05,0.000079,0.000245,0.000006,0.000061,0.000000e+00,0.000052,...,0.000018,0.000078,0.000070,0.000116,0.000078,0.000061,0.000102,0.000021,0.000100,MILD
1,0.000000,0.000000,0.000015,8.515219e-04,0.000107,0.001334,0.005597,0.001388,5.144433e-04,0.001511,...,0.000032,0.000004,0.000012,0.000007,0.000152,0.000008,0.000397,0.000112,0.000003,MILD
2,0.000210,0.000006,0.000018,8.649685e-09,0.000084,0.000487,0.000185,0.000092,1.294411e-04,0.000000,...,0.000119,0.000052,0.000205,0.000420,0.000060,0.000096,0.000243,0.000036,0.000253,MILD
3,0.000171,0.000233,0.000076,4.854921e-05,0.000122,0.000333,0.000038,0.000075,1.764229e-07,0.000064,...,0.000134,0.000038,0.000153,0.000124,0.000026,0.000012,0.000147,0.000052,0.000086,MILD
4,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000078,0.000074,0.000027,0.000079,0.000018,0.000044,0.000074,0.000042,0.000040,MILD


In [154]:
##### Preprocessing

train_mod = train.copy()
# Imputation using mean method
#train_mod = train_mod.replace(0,np.nan)
#train_mod_imp = train_mod.transform(lambda x: x.fillna(x.mean()))

In [156]:
train_mod.shape

(132, 547)

In [157]:
train_mod.head()

,Freq.1198.26559877594,Freq.1204.07148226988,Freq.1211.13623593336,Freq.1217.73683596229,Freq.1223.16958659828,Freq.1234.45996741201,Freq.1239.41043347512,Freq.1244.8230732813,Freq.1254.64710066874,Freq.1261.3335177338,...,Freq.9059.04784969308,Freq.9098.3102509794,Freq.9436.045671808,Freq.9594.78353572215,Freq.9799.45047805923,Freq.10431.4344537929,Freq.11007.128608859,Freq.11035.3843513154,Freq.11160.836582373,Group
0,0.000251,0.000345,0.000074,3.541783e-05,0.000079,0.000245,0.000006,0.000061,0.000000e+00,0.000052,...,0.000018,0.000078,0.000070,0.000116,0.000078,0.000061,0.000102,0.000021,0.000100,MILD
1,0.000000,0.000000,0.000015,8.515219e-04,0.000107,0.001334,0.005597,0.001388,5.144433e-04,0.001511,...,0.000032,0.000004,0.000012,0.000007,0.000152,0.000008,0.000397,0.000112,0.000003,MILD
2,0.000210,0.000006,0.000018,8.649685e-09,0.000084,0.000487,0.000185,0.000092,1.294411e-04,0.000000,...,0.000119,0.000052,0.000205,0.000420,0.000060,0.000096,0.000243,0.000036,0.000253,MILD
3,0.000171,0.000233,0.000076,4.854921e-05,0.000122,0.000333,0.000038,0.000075,1.764229e-07,0.000064,...,0.000134,0.000038,0.000153,0.000124,0.000026,0.000012,0.000147,0.000052,0.000086,MILD
4,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000078,0.000074,0.000027,0.000079,0.000018,0.000044,0.000074,0.000042,0.000040,MILD


In [158]:
test = pd.read_csv(TEST_CLINICAL_FILENAME, sep=",")
test = test.iloc[: , 1:]
test_mod = test.copy()
test_mod.head()

,Freq.1198.26559877594,Freq.1204.07148226988,Freq.1211.13623593336,Freq.1217.73683596229,Freq.1223.16958659828,Freq.1234.45996741201,Freq.1239.41043347512,Freq.1244.8230732813,Freq.1254.64710066874,Freq.1261.3335177338,...,Freq.9059.04784969308,Freq.9098.3102509794,Freq.9436.045671808,Freq.9594.78353572215,Freq.9799.45047805923,Freq.10431.4344537929,Freq.11007.128608859,Freq.11035.3843513154,Freq.11160.836582373,Group
0,1.363486e-04,0.000205,0.000115,0.000077,0.000199,0.000066,0.000024,0.000406,0.000072,0.000154,...,0.000055,0.000032,0.000028,0.000051,0.000025,0.000056,0.000083,0.000074,0.000035,MILD
1,8.938546e-07,0.000051,0.000017,0.000024,0.000017,0.000072,0.000062,0.000202,0.000217,0.000036,...,0.000026,0.000037,0.000054,0.000500,0.000125,0.000128,0.000049,0.000004,0.000192,MILD
2,2.061797e-04,0.000237,0.000002,0.000109,0.000136,0.000029,0.000629,0.000150,0.000105,0.000229,...,0.000135,0.000079,0.000053,0.000232,0.000076,0.000187,0.000120,0.000031,0.000156,MILD
3,2.256595e-03,0.001221,0.000174,0.000109,0.001292,0.000142,0.000579,0.001619,0.000003,0.002916,...,0.000016,0.000010,0.000009,0.000020,0.000030,0.000006,0.000015,0.000018,0.000021,MILD
4,1.023898e-05,0.000314,0.000073,0.000023,0.000071,0.000000,0.000430,0.000378,0.000096,0.000026,...,0.000072,0.000019,0.000072,0.000166,0.000006,0.000204,0.000079,0.000026,0.000094,MILD


In [159]:
X_test, y_test = test_mod.drop("Group", axis=1), test_mod["Group"]

---

### 3) Baseline Model Training and CV

In [160]:
# Define Classifier (or pipeline)
clf = lgb.LGBMClassifier(random_state=SEED)

In [161]:
# Get Features and Target
X, y = train_mod.drop("Group", axis=1), train_mod["Group"]

In [162]:
# Defining RepeatedKFold Cross Validator
rkf = RepeatedKFold(n_splits=5, n_repeats=20, random_state=SEED)

In [163]:
# Define metric scorer
metric_scorer = make_scorer(balanced_accuracy_score)
metric_scorer

make_scorer(balanced_accuracy_score)

In [164]:
# Cross validate model
scores = cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1)

In [165]:
# Score from each CV Iteration
scores

array([0.54605263, 0.94444444, 0.70676692, 0.61538462, 0.7797619 ,
       0.66666667, 0.875     , 0.67857143, 0.75625   , 0.83333333,
       0.64204545, 0.9       , 0.65413534, 0.81944444, 0.6875    ,
       0.80555556, 0.84117647, 0.71428571, 0.85947712, 0.71568627,
       0.725     , 0.63486842, 0.73030303, 0.66875   , 0.8875    ,
       0.76666667, 0.76923077, 0.65972222, 0.7745098 , 0.66013072,
       0.61111111, 0.73235294, 0.87593985, 0.71875   , 0.91176471,
       0.77272727, 0.66118421, 0.7202381 , 0.625     , 0.85714286,
       0.77777778, 0.875     , 0.67272727, 0.81818182, 0.69444444,
       0.89285714, 0.68181818, 0.74509804, 0.76875   , 0.89473684,
       0.84117647, 0.68333333, 0.8875    , 0.68954248, 0.65625   ,
       0.78693182, 0.76176471, 0.57638889, 0.79761905, 0.90625   ,
       0.81176471, 0.75      , 0.7202381 , 0.63194444, 0.72619048,
       0.725     , 0.8       , 0.90972222, 0.65972222, 0.73939394,
       0.84659091, 0.75      , 0.9       , 0.74183007, 0.75151

In [166]:
# Mean Metric Value
np.mean(scores)

0.7530414906515912

# Feature reduction

In [167]:
train_mod_stats = train_mod.drop('Group', axis = 1)


In [168]:
df = train_mod.copy()
peaks_list = []
pval_list = []
df2 = pd.melt(df, id_vars = ["Group"])
mild = df.where(df.Group == "MILD").dropna()
sev = df.where(df.Group == "SEVERE").dropna()
for i in range(1,(len(df.columns)-1)):
    s, p1 = stats.mannwhitneyu(mild.iloc[:,i], sev.iloc[:,i], alternative = 'two-sided')
    if p1 <= 0.05:
        pval_list.append(p1)
        peaks_list.append(df.columns[i])
peaks_list.append("Group")

In [169]:
df_filtered = df.drop(columns=[col for col in df if col not in peaks_list])
df_test_filtered = test.drop(columns=[col for col in test if col not in peaks_list])

In [170]:
df_test_filtered.shape

(64, 119)

---

In [171]:
# Get Features and Target
X, y = df_filtered.drop("Group", axis=1), df_filtered["Group"]
X_test, y_test = df_test_filtered.drop("Group", axis=1), df_test_filtered["Group"]
# Cross validate model
scores = cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1)
# Score from each CV Iteration
scores

array([0.63486842, 0.97222222, 0.70676692, 0.73076923, 0.66666667,
       0.74166667, 1.        , 0.72619048, 0.80625   , 0.83333333,
       0.6875    , 1.        , 0.77819549, 0.76388889, 0.76875   ,
       0.94444444, 0.84117647, 0.78571429, 0.85947712, 0.74509804,
       0.88333333, 0.72368421, 0.77575758, 0.81875   , 0.8875    ,
       0.84166667, 0.92307692, 0.63194444, 0.77124183, 0.63398693,
       0.69444444, 0.75294118, 0.70676692, 0.6875    , 0.88235294,
       0.83238636, 0.69736842, 0.76190476, 0.75833333, 0.92857143,
       0.77777778, 0.875     , 0.69393939, 0.83030303, 0.78472222,
       0.85714286, 0.74147727, 0.74509804, 0.8375    , 0.89473684,
       0.78235294, 0.80833333, 0.8875    , 0.74509804, 0.65625   ,
       0.81534091, 0.76176471, 0.63888889, 0.76190476, 0.96875   ,
       0.9       , 0.77777778, 0.76190476, 0.78472222, 0.76785714,
       0.69166667, 0.8       , 0.9375    , 0.75694444, 0.87575758,
       0.87784091, 0.8       , 0.7875    , 0.74183007, 0.78484

In [172]:
# Mean Metric Value
np.mean(scores)

0.7912739915743012

### 4) Experiments

##### 4.1) Hyper Parameter Optimization with Optuna

In [173]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model with param trial
    clf = lgb.LGBMClassifier(random_state=SEED, **param)
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [174]:
study = optuna.create_study(direction="maximize")

[I 2022-06-23 23:34:05,756] A new study created in memory with name: no-name-266a551f-14d7-4c9e-bae0-eb3eeb77289c


In [175]:
study.optimize(objective, n_trials=500)

[I 2022-06-23 23:34:06,946] Trial 0 finished with value: 0.5 and parameters: {'lambda_l1': 2.3595784575679856e-05, 'lambda_l2': 1.1012063752489056e-06, 'num_leaves': 184, 'feature_fraction': 0.6866384687759279, 'bagging_fraction': 0.6995098509208947, 'bagging_freq': 7, 'min_child_samples': 75, 'max_depth': 294}. Best is trial 0 with value: 0.5.
[I 2022-06-23 23:34:08,520] Trial 1 finished with value: 0.7823621184027532 and parameters: {'lambda_l1': 3.8693967443574304, 'lambda_l2': 9.689284707975446e-08, 'num_leaves': 180, 'feature_fraction': 0.8189516075958034, 'bagging_fraction': 0.9077794004028834, 'bagging_freq': 1, 'min_child_samples': 48, 'max_depth': 906}. Best is trial 1 with value: 0.7823621184027532.
[I 2022-06-23 23:34:09,175] Trial 2 finished with value: 0.5 and parameters: {'lambda_l1': 0.04583950873771014, 'lambda_l2': 0.030289944481967472, 'num_leaves': 158, 'feature_fraction': 0.4396819945927838, 'bagging_fraction': 0.6935136647082449, 'bagging_freq': 6, 'min_child_sampl

[I 2022-06-23 23:34:32,041] Trial 22 finished with value: 0.8458864038653124 and parameters: {'lambda_l1': 0.0049848585689571435, 'lambda_l2': 0.4350407031813399, 'num_leaves': 190, 'feature_fraction': 0.40232254850151916, 'bagging_fraction': 0.7686444240928251, 'bagging_freq': 5, 'min_child_samples': 20, 'max_depth': 191}. Best is trial 20 with value: 0.8514517096680408.
[I 2022-06-23 23:34:32,944] Trial 23 finished with value: 0.8483214730960473 and parameters: {'lambda_l1': 0.00014041839015897656, 'lambda_l2': 0.024737071573333977, 'num_leaves': 141, 'feature_fraction': 0.5309675943408307, 'bagging_fraction': 0.6007980823669852, 'bagging_freq': 4, 'min_child_samples': 32, 'max_depth': 324}. Best is trial 20 with value: 0.8514517096680408.
[I 2022-06-23 23:34:34,448] Trial 24 finished with value: 0.8261507213546686 and parameters: {'lambda_l1': 0.3387155633074678, 'lambda_l2': 0.00013554409961060858, 'num_leaves': 103, 'feature_fraction': 0.5148044341195899, 'bagging_fraction': 0.828

[I 2022-06-23 23:34:56,458] Trial 44 finished with value: 0.8309216784462528 and parameters: {'lambda_l1': 0.12805272165280263, 'lambda_l2': 1.1096030783653459e-08, 'num_leaves': 146, 'feature_fraction': 0.564379479523821, 'bagging_fraction': 0.4740626746323659, 'bagging_freq': 7, 'min_child_samples': 14, 'max_depth': 160}. Best is trial 34 with value: 0.8587185535482749.
[I 2022-06-23 23:34:58,260] Trial 45 finished with value: 0.847702398521671 and parameters: {'lambda_l1': 0.0042033404046229175, 'lambda_l2': 3.45172810622433e-08, 'num_leaves': 82, 'feature_fraction': 0.5986836272784102, 'bagging_fraction': 0.5004357529230422, 'bagging_freq': 3, 'min_child_samples': 7, 'max_depth': 295}. Best is trial 34 with value: 0.8587185535482749.
[I 2022-06-23 23:34:59,397] Trial 46 finished with value: 0.8450055008416075 and parameters: {'lambda_l1': 0.6769662413006188, 'lambda_l2': 1.2202155135645877e-07, 'num_leaves': 128, 'feature_fraction': 0.6577406849774157, 'bagging_fraction': 0.4587901

[I 2022-06-23 23:35:23,777] Trial 66 finished with value: 0.8624388564222155 and parameters: {'lambda_l1': 3.332194359867164e-08, 'lambda_l2': 2.8748091577579024e-07, 'num_leaves': 99, 'feature_fraction': 0.5202959488420069, 'bagging_fraction': 0.6412478137487818, 'bagging_freq': 4, 'min_child_samples': 31, 'max_depth': 890}. Best is trial 62 with value: 0.8627685659283957.
[I 2022-06-23 23:35:25,717] Trial 67 finished with value: 0.8590302445018003 and parameters: {'lambda_l1': 6.486994021115984e-08, 'lambda_l2': 3.3498734276185855e-07, 'num_leaves': 80, 'feature_fraction': 0.5212812170695712, 'bagging_fraction': 0.6395867937595023, 'bagging_freq': 4, 'min_child_samples': 18, 'max_depth': 875}. Best is trial 62 with value: 0.8627685659283957.
[I 2022-06-23 23:35:27,604] Trial 68 finished with value: 0.861898707029125 and parameters: {'lambda_l1': 4.5346043038858475e-08, 'lambda_l2': 2.3372156525239048e-07, 'num_leaves': 71, 'feature_fraction': 0.5274956655946123, 'bagging_fraction': 0

[I 2022-06-23 23:36:01,954] Trial 88 finished with value: 0.8526266152578073 and parameters: {'lambda_l1': 4.818120766385874e-08, 'lambda_l2': 1.2497593771243688e-05, 'num_leaves': 47, 'feature_fraction': 0.48639136083604334, 'bagging_fraction': 0.555751654860862, 'bagging_freq': 4, 'min_child_samples': 32, 'max_depth': 914}. Best is trial 62 with value: 0.8627685659283957.
[I 2022-06-23 23:36:02,743] Trial 89 finished with value: 0.5 and parameters: {'lambda_l1': 1.600771207270657e-08, 'lambda_l2': 2.466229149104665e-06, 'num_leaves': 102, 'feature_fraction': 0.5074234648852856, 'bagging_fraction': 0.7060544958260057, 'bagging_freq': 3, 'min_child_samples': 51, 'max_depth': 745}. Best is trial 62 with value: 0.8627685659283957.
[I 2022-06-23 23:36:03,568] Trial 90 finished with value: 0.652139272814389 and parameters: {'lambda_l1': 0.0007686671648022217, 'lambda_l2': 5.509232029833948e-06, 'num_leaves': 21, 'feature_fraction': 0.4410034475085249, 'bagging_fraction': 0.6756921246120623

...

[I 2022-06-23 23:46:01,082] Trial 411 finished with value: 0.8621527785295199 and parameters: {'lambda_l1': 7.862166665867306e-07, 'lambda_l2': 9.375295112829582e-07, 'num_leaves': 149, 'feature_fraction': 0.5195885221499709, 'bagging_fraction': 0.5074669680301662, 'bagging_freq': 7, 'min_child_samples': 24, 'max_depth': 1000}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:46:03,744] Trial 412 finished with value: 0.8485060990136453 and parameters: {'lambda_l1': 4.4308282282747516e-07, 'lambda_l2': 1.2717635615583436e-07, 'num_leaves': 185, 'feature_fraction': 0.5131652890535878, 'bagging_fraction': 0.8314598201254372, 'bagging_freq': 7, 'min_child_samples': 25, 'max_depth': 955}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:46:06,098] Trial 413 finished with value: 0.8655736146378561 and parameters: {'lambda_l1': 1.2132822998094689e-06, 'lambda_l2': 1.972375816184608e-06, 'num_leaves': 152, 'feature_fraction': 0.539245041044746, 'bagging_frac

[I 2022-06-23 23:46:42,814] Trial 432 finished with value: 0.8587951116006457 and parameters: {'lambda_l1': 1.4618734850486864e-06, 'lambda_l2': 2.785306771823844e-07, 'num_leaves': 203, 'feature_fraction': 0.4398351627047236, 'bagging_fraction': 0.5587211703943287, 'bagging_freq': 7, 'min_child_samples': 30, 'max_depth': 885}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:46:45,113] Trial 433 finished with value: 0.85830191340737 and parameters: {'lambda_l1': 2.51236434437743e-07, 'lambda_l2': 1.8500154137947161e-07, 'num_leaves': 124, 'feature_fraction': 0.4258237180209296, 'bagging_fraction': 0.5485874266513936, 'bagging_freq': 3, 'min_child_samples': 27, 'max_depth': 999}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:46:45,677] Trial 434 finished with value: 0.5 and parameters: {'lambda_l1': 1.0636700836231468e-06, 'lambda_l2': 4.381405214615813e-07, 'num_leaves': 17, 'feature_fraction': 0.7342794959526207, 'bagging_fraction': 0.5269836892

[I 2022-06-23 23:47:33,652] Trial 454 finished with value: 0.8635215630602627 and parameters: {'lambda_l1': 4.3002192248139256e-07, 'lambda_l2': 0.028597780748577294, 'num_leaves': 110, 'feature_fraction': 0.48550049305664245, 'bagging_fraction': 0.525828965467082, 'bagging_freq': 7, 'min_child_samples': 28, 'max_depth': 956}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:47:36,845] Trial 455 finished with value: 0.8533881371251725 and parameters: {'lambda_l1': 6.370676102521838e-08, 'lambda_l2': 0.027683053584809363, 'num_leaves': 98, 'feature_fraction': 0.44052853721318935, 'bagging_fraction': 0.7429723414731342, 'bagging_freq': 7, 'min_child_samples': 23, 'max_depth': 977}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:47:39,288] Trial 456 finished with value: 0.8604125516511353 and parameters: {'lambda_l1': 9.679893940325626e-07, 'lambda_l2': 0.0798298504459157, 'num_leaves': 128, 'feature_fraction': 0.46269500166706584, 'bagging_fraction':

[I 2022-06-23 23:48:21,140] Trial 475 finished with value: 0.8584953768926291 and parameters: {'lambda_l1': 2.4852837200631695e-07, 'lambda_l2': 2.3422306931688404e-06, 'num_leaves': 90, 'feature_fraction': 0.41176162863497945, 'bagging_fraction': 0.5597154304705471, 'bagging_freq': 6, 'min_child_samples': 22, 'max_depth': 850}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:48:23,256] Trial 476 finished with value: 0.8573552238310365 and parameters: {'lambda_l1': 6.555638964960354e-07, 'lambda_l2': 3.945930538524973e-06, 'num_leaves': 95, 'feature_fraction': 0.42314432709406696, 'bagging_fraction': 0.5729497265985805, 'bagging_freq': 7, 'min_child_samples': 18, 'max_depth': 801}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:48:25,359] Trial 477 finished with value: 0.8652563879404049 and parameters: {'lambda_l1': 8.007413607892195e-07, 'lambda_l2': 8.305079310786386e-07, 'num_leaves': 106, 'feature_fraction': 0.43352943884006595, 'bagging_frac

[I 2022-06-23 23:49:05,827] Trial 496 finished with value: 0.8651252549050535 and parameters: {'lambda_l1': 0.00021724973622386085, 'lambda_l2': 3.213181290608168e-07, 'num_leaves': 89, 'feature_fraction': 0.43212384421408406, 'bagging_fraction': 0.5437777658203073, 'bagging_freq': 7, 'min_child_samples': 29, 'max_depth': 766}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:49:08,503] Trial 497 finished with value: 0.8631671972248597 and parameters: {'lambda_l1': 0.00936132840023022, 'lambda_l2': 1.9074403363786395e-07, 'num_leaves': 148, 'feature_fraction': 0.46194213370316994, 'bagging_fraction': 0.5589369795428327, 'bagging_freq': 7, 'min_child_samples': 26, 'max_depth': 848}. Best is trial 389 with value: 0.8707509203687609.
[I 2022-06-23 23:49:09,761] Trial 498 finished with value: 0.6010329444162649 and parameters: {'lambda_l1': 4.6622755268778554e-07, 'lambda_l2': 1.0941142690987427e-06, 'num_leaves': 105, 'feature_fraction': 0.4952329137180995, 'bagging_frac

In [176]:
# Get best trial based on metric score
trial = study.best_trial

In [177]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    lambda_l1: 1.2537265730755262e-06
    lambda_l2: 3.451074192111965e-07
    num_leaves: 141
    feature_fraction: 0.4703062413129908
    bagging_fraction: 0.5227902554474249
    bagging_freq: 7
    min_child_samples: 27
    max_depth: 996


In [178]:
# Best Score from HP Opt
trial.values[0]

0.8707509203687609

- Aumento de performance considerável em relação ao valor baseline para um LGBM

##### 4.2) PCA Dimension Reduction + Hyper Parameter Optimization with Optuna

In [181]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # PCA Parameter Grid
    pca_param = {
        "n_components": trial.suggest_int("n_components", 5, 100)
    }
    # Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("pca", PCA(**pca_param)),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [182]:
study = optuna.create_study(direction="maximize")

[I 2022-06-17 15:30:06,080] A new study created in memory with name: no-name-adfd880b-812e-41f4-828a-f1b968db75c6


In [ ]:
study.optimize(objective, n_trials=500)

In [26]:
# Get best trial based on metric score
trial = study.best_trial

In [27]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    n_components: 98
    lambda_l1: 4.997984938308335e-06
    lambda_l2: 9.51579276153768e-06
    num_leaves: 44
    feature_fraction: 0.5421454819677247
    bagging_fraction: 0.48235662397332973
    bagging_freq: 4
    min_child_samples: 8
    max_depth: 563


In [28]:
# Best Score from HP Opt
trial.values[0]

0.7134464057614213

- Utilizar redução de dimensionalidade via PCA piorou bastante a performance do modelo

##### 4.3) Features Scaler + Hyper Parameter Optimization with Optuna

In [29]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # List scalers to chose from
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])
    # Define scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()
    # LGBM Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("scaler", scaler),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [30]:
study = optuna.create_study(direction="maximize")

[I 2022-06-07 22:15:09,043] A new study created in memory with name: no-name-9d027bd4-11fc-4b2d-8846-da6f6ffb30f6


In [ ]:
study.optimize(objective, n_trials=500)

In [32]:
# Get best trial based on metric score
trial = study.best_trial

In [33]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    scalers: robust
    lambda_l1: 4.3223282482835764e-07
    lambda_l2: 3.3796186878776776e-05
    num_leaves: 188
    feature_fraction: 0.650978565866076
    bagging_fraction: 0.9212536490874126
    bagging_freq: 7
    min_child_samples: 36
    max_depth: 956


In [34]:
# Best Score from HP Opt
trial.values[0]

0.8521054248039541

- A adição de um estágio de feature scaling antes do treinamento parece ter ajudado o modelo

##### 4.4) Boruta Feature Selection + Features Scaler + Hyper Parameter Optimization with Optuna

In [217]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=SEED, )

In [218]:
feat_selector.fit(np.array(X), y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	70
Rejected: 	476
Iteration: 	9 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	10 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	11 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	12 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	13 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	14 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	15 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	16 / 100
Confirmed: 	2

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=193, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x1BC734BF140),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1BC734BF140, verbose=2)

In [219]:
print("Number of selected features: ", feat_selector.n_features_)

Number of selected features:  33


In [220]:
# Best Features (accord to Boruta)
X.columns[feat_selector.support_]

Index(['Freq.1399.46591504505', 'Freq.1522.99914751846',
       'Freq.1715.83254187774', 'Freq.1740.12061901788',
       'Freq.1794.21713030157', 'Freq.2032.98713905056',
       'Freq.2100.44990262345', 'Freq.2182.54012190969',
       'Freq.2187.26929655148', 'Freq.2241.01398322552',
       'Freq.2461.57721259156', 'Freq.2822.17822957638',
       'Freq.2981.05105455515', 'Freq.3083.87231952593',
       'Freq.3795.45160708473', 'Freq.3983.61553339652',
       'Freq.4266.97846896688', 'Freq.4283.95182164633',
       'Freq.4307.03317519015', 'Freq.4395.11277752994',
       'Freq.4495.09063766933', 'Freq.4659.55667096198',
       'Freq.4802.98802524845', 'Freq.4823.08140765752',
       'Freq.5084.14952111257', 'Freq.5224.84719303067',
       'Freq.5272.6322523475', 'Freq.5433.53206707083',
       'Freq.5485.69282171011', 'Freq.5720.79450801948',
       'Freq.7738.2889532685', 'Freq.8943.8000787644', 'Freq.9098.3102509794'],
      dtype='object')

In [221]:
# Filter most importante features
X_transform = feat_selector.transform(np.array(X))

In [222]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # List scalers to chose from
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])
    # Define scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()
    # LGBM Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("scaler", scaler),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X_transform, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [223]:
study = optuna.create_study(direction="maximize")

[I 2022-06-24 00:05:09,001] A new study created in memory with name: no-name-7e03f0db-a45d-43fa-aeb6-3eaeb5815ff6


In [224]:
study.optimize(objective, n_trials=500)

[I 2022-06-24 00:07:07,780] Trial 0 finished with value: 0.5 and parameters: {'scalers': 'standard', 'lambda_l1': 1.736164714022951e-05, 'lambda_l2': 0.0008744806520391302, 'num_leaves': 186, 'feature_fraction': 0.8593730467230957, 'bagging_fraction': 0.7039252573471362, 'bagging_freq': 5, 'min_child_samples': 67, 'max_depth': 291}. Best is trial 0 with value: 0.5.
[I 2022-06-24 00:07:08,583] Trial 1 finished with value: 0.8603665093506424 and parameters: {'scalers': 'standard', 'lambda_l1': 0.008622697977459544, 'lambda_l2': 6.208314622454566e-08, 'num_leaves': 184, 'feature_fraction': 0.701051773780067, 'bagging_fraction': 0.4008763757819356, 'bagging_freq': 1, 'min_child_samples': 20, 'max_depth': 476}. Best is trial 1 with value: 0.8603665093506424.
[I 2022-06-24 00:07:09,649] Trial 2 finished with value: 0.8380926025728658 and parameters: {'scalers': 'robust', 'lambda_l1': 0.17266781673396256, 'lambda_l2': 1.059988995841295e-07, 'num_leaves': 30, 'feature_fraction': 0.478233073255

[I 2022-06-24 00:07:27,143] Trial 21 finished with value: 0.8489316654063171 and parameters: {'scalers': 'standard', 'lambda_l1': 1.402975481383046e-08, 'lambda_l2': 9.991701764028775e-05, 'num_leaves': 225, 'feature_fraction': 0.9238790545409372, 'bagging_fraction': 0.47278969400458437, 'bagging_freq': 3, 'min_child_samples': 18, 'max_depth': 562}. Best is trial 18 with value: 0.8606703089007656.
[I 2022-06-24 00:07:28,404] Trial 22 finished with value: 0.8557324019289964 and parameters: {'scalers': 'standard', 'lambda_l1': 1.9395373152078576e-07, 'lambda_l2': 2.458075416311321e-07, 'num_leaves': 227, 'feature_fraction': 0.9393163921111374, 'bagging_fraction': 0.45858041879466954, 'bagging_freq': 2, 'min_child_samples': 18, 'max_depth': 556}. Best is trial 18 with value: 0.8606703089007656.
[I 2022-06-24 00:07:29,803] Trial 23 finished with value: 0.8553611502231858 and parameters: {'scalers': 'standard', 'lambda_l1': 1.023051122687271e-07, 'lambda_l2': 2.0728780087572496e-05, 'num_le

[I 2022-06-24 00:07:43,881] Trial 42 finished with value: 0.5 and parameters: {'scalers': 'minmax', 'lambda_l1': 4.016506692881616e-08, 'lambda_l2': 8.445820809561303e-06, 'num_leaves': 159, 'feature_fraction': 0.9554397718072419, 'bagging_fraction': 0.7313497446825487, 'bagging_freq': 1, 'min_child_samples': 46, 'max_depth': 401}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:07:44,462] Trial 43 finished with value: 0.8532307688656373 and parameters: {'scalers': 'robust', 'lambda_l1': 3.3006575930054615e-07, 'lambda_l2': 4.0587916862520554e-05, 'num_leaves': 144, 'feature_fraction': 0.8524526137899114, 'bagging_fraction': 0.6827111778722371, 'bagging_freq': 2, 'min_child_samples': 31, 'max_depth': 138}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:07:45,844] Trial 44 finished with value: 0.8565074724252356 and parameters: {'scalers': 'robust', 'lambda_l1': 1.718341135731164e-08, 'lambda_l2': 1.010455807137174e-06, 'num_leaves': 85, 'feature_fract

[I 2022-06-24 00:08:01,232] Trial 63 finished with value: 0.8597860226074237 and parameters: {'scalers': 'standard', 'lambda_l1': 2.640054204056451e-08, 'lambda_l2': 2.10337760151054e-08, 'num_leaves': 184, 'feature_fraction': 0.7129508948932183, 'bagging_fraction': 0.4052925742915207, 'bagging_freq': 1, 'min_child_samples': 22, 'max_depth': 665}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:08:01,819] Trial 64 finished with value: 0.5 and parameters: {'scalers': 'standard', 'lambda_l1': 5.72724282074693e-08, 'lambda_l2': 1.6165174186927202e-07, 'num_leaves': 157, 'feature_fraction': 0.7290473533412141, 'bagging_fraction': 0.4385790711428784, 'bagging_freq': 1, 'min_child_samples': 32, 'max_depth': 750}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:08:02,303] Trial 65 finished with value: 0.5 and parameters: {'scalers': 'standard', 'lambda_l1': 2.358237908871582e-08, 'lambda_l2': 3.9888316362284076e-07, 'num_leaves': 142, 'feature_fraction': 0.76

[I 2022-06-24 00:08:18,053] Trial 84 finished with value: 0.8640526208182941 and parameters: {'scalers': 'robust', 'lambda_l1': 8.569444484836788e-08, 'lambda_l2': 0.003779939269437846, 'num_leaves': 175, 'feature_fraction': 0.48316712380251475, 'bagging_fraction': 0.5208280746556682, 'bagging_freq': 1, 'min_child_samples': 19, 'max_depth': 309}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:08:18,612] Trial 85 finished with value: 0.7416672635500035 and parameters: {'scalers': 'robust', 'lambda_l1': 1.5643004554794204e-08, 'lambda_l2': 0.001078940432489709, 'num_leaves': 176, 'feature_fraction': 0.9055578204624273, 'bagging_fraction': 0.40177537631439764, 'bagging_freq': 1, 'min_child_samples': 26, 'max_depth': 311}. Best is trial 24 with value: 0.870197741929553.
[I 2022-06-24 00:08:19,400] Trial 86 finished with value: 0.8526485367702473 and parameters: {'scalers': 'robust', 'lambda_l1': 4.340889648922245e-08, 'lambda_l2': 0.0516618407255378, 'num_leaves': 170, 'f

...

[I 2022-06-24 00:13:44,203] Trial 415 finished with value: 0.867273502361931 and parameters: {'scalers': 'robust', 'lambda_l1': 4.567564657823748e-07, 'lambda_l2': 1.0309303053758623, 'num_leaves': 136, 'feature_fraction': 0.8559907488727889, 'bagging_fraction': 0.4532189792511736, 'bagging_freq': 1, 'min_child_samples': 18, 'max_depth': 312}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:13:45,144] Trial 416 finished with value: 0.855779762033632 and parameters: {'scalers': 'standard', 'lambda_l1': 1.3866430297754027e-06, 'lambda_l2': 0.49330349439851795, 'num_leaves': 27, 'feature_fraction': 0.9454132984200015, 'bagging_fraction': 0.5544379225831841, 'bagging_freq': 2, 'min_child_samples': 12, 'max_depth': 274}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:13:46,041] Trial 417 finished with value: 0.8415174342556309 and parameters: {'scalers': 'robust', 'lambda_l1': 6.727195594522539e-07, 'lambda_l2': 0.28977651737329996, 'num_leaves': 119, 

[I 2022-06-24 00:14:03,030] Trial 436 finished with value: 0.8601204843273342 and parameters: {'scalers': 'robust', 'lambda_l1': 1.1174549342978452e-07, 'lambda_l2': 2.2714201909080423, 'num_leaves': 233, 'feature_fraction': 0.9135453525812575, 'bagging_fraction': 0.6408961740140573, 'bagging_freq': 2, 'min_child_samples': 21, 'max_depth': 381}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:03,844] Trial 437 finished with value: 0.8657390188715653 and parameters: {'scalers': 'standard', 'lambda_l1': 4.812127031401299e-06, 'lambda_l2': 0.7689851018072759, 'num_leaves': 30, 'feature_fraction': 0.9371356197225286, 'bagging_fraction': 0.4872149732209391, 'bagging_freq': 1, 'min_child_samples': 19, 'max_depth': 347}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:04,668] Trial 438 finished with value: 0.8667566041199943 and parameters: {'scalers': 'robust', 'lambda_l1': 1.6186092503713917e-06, 'lambda_l2': 0.49335681485803484, 'num_leaves': 105

[I 2022-06-24 00:14:19,392] Trial 456 finished with value: 0.8628238760938065 and parameters: {'scalers': 'robust', 'lambda_l1': 5.669313556366672e-07, 'lambda_l2': 1.0848805461650514e-07, 'num_leaves': 24, 'feature_fraction': 0.9298998657760505, 'bagging_fraction': 0.48393097305090405, 'bagging_freq': 1, 'min_child_samples': 22, 'max_depth': 351}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:19,823] Trial 457 finished with value: 0.7569378375752216 and parameters: {'scalers': 'standard', 'lambda_l1': 1.2873625751923197e-07, 'lambda_l2': 0.06055678155514418, 'num_leaves': 108, 'feature_fraction': 0.9165123407076817, 'bagging_fraction': 0.4007256886105837, 'bagging_freq': 2, 'min_child_samples': 26, 'max_depth': 294}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:20,774] Trial 458 finished with value: 0.8656200601084503 and parameters: {'scalers': 'robust', 'lambda_l1': 1.728333673601931e-06, 'lambda_l2': 0.11017201859345915, 'num_leaves'

[I 2022-06-24 00:14:35,313] Trial 476 finished with value: 0.8525593227506926 and parameters: {'scalers': 'standard', 'lambda_l1': 3.655023538203521e-07, 'lambda_l2': 1.8925273785925903e-05, 'num_leaves': 35, 'feature_fraction': 0.9491278433284144, 'bagging_fraction': 0.5270579409131134, 'bagging_freq': 1, 'min_child_samples': 5, 'max_depth': 276}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:36,156] Trial 477 finished with value: 0.8645600994507728 and parameters: {'scalers': 'robust', 'lambda_l1': 1.2159032131384854e-07, 'lambda_l2': 0.6569944434810124, 'num_leaves': 96, 'feature_fraction': 0.8606045208298949, 'bagging_fraction': 0.5022703561071765, 'bagging_freq': 2, 'min_child_samples': 23, 'max_depth': 333}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:36,808] Trial 478 finished with value: 0.8663079462676986 and parameters: {'scalers': 'standard', 'lambda_l1': 3.3678297103803053e-06, 'lambda_l2': 0.9269931307957615, 'num_leaves': 

[I 2022-06-24 00:14:54,935] Trial 497 finished with value: 0.8663799573749266 and parameters: {'scalers': 'robust', 'lambda_l1': 5.427335872538109e-07, 'lambda_l2': 1.7704879606874941, 'num_leaves': 57, 'feature_fraction': 0.8610745154107482, 'bagging_fraction': 0.4319905870341003, 'bagging_freq': 1, 'min_child_samples': 12, 'max_depth': 480}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:55,999] Trial 498 finished with value: 0.8557544824886156 and parameters: {'scalers': 'robust', 'lambda_l1': 8.556695614332493e-07, 'lambda_l2': 0.00021356760518056062, 'num_leaves': 51, 'feature_fraction': 0.8284412766724575, 'bagging_fraction': 0.448774866217654, 'bagging_freq': 1, 'min_child_samples': 16, 'max_depth': 406}. Best is trial 292 with value: 0.8718675463897988.
[I 2022-06-24 00:14:56,990] Trial 499 finished with value: 0.8582455141178054 and parameters: {'scalers': 'robust', 'lambda_l1': 1.3118950510214943e-06, 'lambda_l2': 4.6606338141436545, 'num_leaves': 53, '

In [225]:
# Get best trial based on metric score
trial = study.best_trial

In [226]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    scalers: standard
    lambda_l1: 1.580293918334291e-07
    lambda_l2: 0.0004709721620374175
    num_leaves: 31
    feature_fraction: 0.9644780321403139
    bagging_fraction: 0.5249470583723804
    bagging_freq: 1
    min_child_samples: 27
    max_depth: 319


In [227]:
# Best Score from HP Opt
trial.values[0]

0.8718675463897988

In [228]:
gbm_opt = lgb.LGBMClassifier(random_state=SEED, **trial.params)

- Pré selecionar as features mais importantes auxiliou a obter uma performance ainda melhor a partir do pipeline com Robust Scaler e LightGBM

In [229]:
X_transform_test = feat_selector.transform(np.array(X_test))

In [230]:
X_transform_test.shape

(64, 33)

In [231]:
gbm_opt.fit(X_transform,y)

[LightGBM] [Warning] Unknown parameter: scalers
[LightGBM] [Warning] feature_fraction is set=0.9644780321403139, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9644780321403139
[LightGBM] [Warning] lambda_l1 is set=1.580293918334291e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.580293918334291e-07
[LightGBM] [Warning] bagging_fraction is set=0.5249470583723804, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5249470583723804
[LightGBM] [Warning] lambda_l2 is set=0.0004709721620374175, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0004709721620374175
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMClassifier(bagging_fraction=0.5249470583723804, bagging_freq=1,
               feature_fraction=0.9644780321403139,
               lambda_l1=1.580293918334291e-07, lambda_l2=0.0004709721620374175,
               max_depth=319, min_child_samples=27, random_state=42,
               scalers='standard')

In [232]:
y_pred = gbm_opt.predict(X_transform_test)

In [233]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.72      0.81      0.76        42
      SEVERE       0.53      0.41      0.46        22

    accuracy                           0.67        64
   macro avg       0.63      0.61      0.61        64
weighted avg       0.66      0.67      0.66        64



### Testing other algorithms 

In [179]:
X_test, y_test = test_mod.drop("Group", axis=1), test_mod["Group"]
X, y = train_mod.drop("Group", axis=1), train_mod["Group"]

###### Get most important features from boruta

In [180]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=SEED, )
feat_selector.fit(np.array(X), y)
print("Number of selected features: ", feat_selector.n_features_)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	546
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	70
Rejected: 	476
Iteration: 	9 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	10 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	11 / 100
Confirmed: 	22
Tentative: 	48
Rejected: 	476
Iteration: 	12 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	13 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	14 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	15 / 100
Confirmed: 	25
Tentative: 	36
Rejected: 	485
Iteration: 	16 / 100
Confirmed: 	2

In [181]:
# Filter most importante features
X_transform = feat_selector.transform(np.array(X))
X_transform_test = feat_selector.transform(np.array(X_test))

In [182]:
gbc = GradientBoostingClassifier(random_state = SEED)

In [183]:
parameters = {'max_depth':list(range(1,100,1)), 'subsample':list(np.arange(0.05,1,0.05)),
             'learning_rate':list(np.arange(0.05,1,0.05)), 'n_estimators':list(range(50,500,25)), }

In [184]:
rscv_gbc = RandomizedSearchCV(gbc, parameters, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 200)

In [185]:
scores_gbc = rscv_gbc.fit(X,y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [186]:
scores_gbc.best_score_

0.7781216577540107

In [187]:
y_pred = scores_gbc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.71      0.71      0.71        42
      SEVERE       0.45      0.45      0.45        22

    accuracy                           0.62        64
   macro avg       0.58      0.58      0.58        64
weighted avg       0.62      0.62      0.62        64



In [188]:
scores_gbc_boruta = rscv_gbc.fit(X_transform, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [189]:
scores_gbc_boruta.best_score_

0.844659090909091

In [190]:
scores_gbc_boruta.best_params_

{'subsample': 0.55,
 'n_estimators': 250,
 'max_depth': 56,
 'learning_rate': 0.9000000000000001}

In [191]:
y_pred = scores_gbc_boruta.predict(X_transform_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.68      0.62      0.65        42
      SEVERE       0.38      0.45      0.42        22

    accuracy                           0.56        64
   macro avg       0.53      0.54      0.53        64
weighted avg       0.58      0.56      0.57        64



In [192]:
lgbm = lgb.LGBMClassifier(random_state=SEED)
parameters = {'num_leaves':list(range(20,100,5)), 'min_child_samples':list(range(5,30,5)),'max_depth':list(range(-1,20,1)),
             'learning_rate':list(np.arange(0.05,1,0.05)),'reg_alpha':list(np.arange(0,0.6, 0.1))}
rscv_lgbm = RandomizedSearchCV(lgbm, parameters, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 200)

In [193]:
scores_lgbm = rscv_lgbm.fit(X, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [194]:
scores_lgbm.best_score_

0.7836363636363636

In [195]:
X_test.shape

(64, 546)

In [196]:
y_pred = scores_lgbm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.70      0.74      0.72        42
      SEVERE       0.45      0.41      0.43        22

    accuracy                           0.62        64
   macro avg       0.58      0.57      0.57        64
weighted avg       0.62      0.62      0.62        64



In [197]:
scores_lgbm_boruta = rscv_lgbm.fit(X_transform, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [198]:
scores_lgbm_boruta.best_score_

0.8555949197860961

In [199]:
X_transform_test.shape

(64, 33)

In [200]:
y_pred = scores_lgbm_boruta.predict(X_transform_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.66      0.64      0.65        42
      SEVERE       0.35      0.36      0.36        22

    accuracy                           0.55        64
   macro avg       0.50      0.50      0.50        64
weighted avg       0.55      0.55      0.55        64



In [211]:
svc_poly = SVC(random_state=SEED)
#parameters = {'C': list(np.arange(1.0,100,0.25)), 'coef0': list(np.arange(1.0,100,0.25)), 'degree': list(range(1,20,1)),
              #'shrinking': ['True', 'False'], 'class_weight': ['balanced'], 'gamma': ['scale','auto'] }

params = {'kernel': ['rbf'],
          'degree': np.linspace(1, 8, 8),
          'C': np.logspace(-3, 5, 40),
          'gamma': np.logspace(-3, 5, 30),
         'class_weight': ['balanced']}
rsvc_svc_poly = RandomizedSearchCV(svc_poly, params, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 500)

In [212]:
scores = rsvc_svc_poly.fit(X, y)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


In [213]:
scores.best_params_

{'kernel': 'rbf',
 'gamma': 1.0826367338740541,
 'degree': 4.0,
 'class_weight': 'balanced',
 'C': 5878.0160722749115}

In [214]:
y_pred = scores.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.68      0.62      0.65        42
      SEVERE       0.38      0.45      0.42        22

    accuracy                           0.56        64
   macro avg       0.53      0.54      0.53        64
weighted avg       0.58      0.56      0.57        64



In [215]:
scores = rsvc_svc_poly.fit(X_transform, y)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


In [216]:
y_pred = scores.predict(X_transform_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.63      0.52      0.57        42
      SEVERE       0.31      0.41      0.35        22

    accuracy                           0.48        64
   macro avg       0.47      0.47      0.46        64
weighted avg       0.52      0.48      0.50        64

